In [ ]:
# pip install pymongo

In [ ]:
import pymongo
from bson.objectid import ObjectId

In [ ]:
host = 'localhost'
port = 27017
username = None
password = None
dbname = 'immobot-pipeline'
collection_name = 'ebay_ads'
client = pymongo.MongoClient(host=host, port=port, username=username, password=password)
collection = client[dbname].get_collection(collection_name)
collection

In [ ]:
collection.find({'internal_id': {'$gt': '1919704805'}})[0]

In [ ]:
filter = {'_id': {'$gt': ObjectId('61810effc1aac7b3c9ed8d4a')}}
ads = collection.find(filter)
# ads = collection.find({})

In [ ]:
collection.count_documents({'_id': {'$gt': ObjectId('61810effc1aac7b3c9ed8d4a')}})

In [ ]:
# import locale
import re

# locale.setlocale(locale.LC_ALL, 'de_DE')
# money = "$6,150,593.22"
money = "1.350 €"
# locale.atof(money.strip("€"))

re.sub(r'[^\d]', '', money)

In [ ]:
import pandas as pd
import re

# ads = collection.find({'_id': {'$gt': ObjectId('61810effc1aac7b3c9ed8d4a')}})

ads = []
for ad in collection.find(filter):
    ad['_id'] = str(ad['_id'])
    ad['price_raw'] = ad['price']
#     print(ad)
    if re.search(r'Zu verschenken', ad['price']):
        ad['price_comment'] = 'give away'
    elif re.search(r' VB', ad['price']):
        ad['price_comment'] = 'negotiation'
    else:
        ad['price_comment'] = ''
    price = re.sub(r'[^\d]', '', ad['price'])
    ad['price'] = int(price) if price else 0
    ads.append(ad)

pd.DataFrame(ads)

In [ ]:
host = 'localhost'
port = 5432
database = 'immobot-pipeline'
user = 'week6rff-pipeline-user'
password = 'week6rff-pipeline-password'
# user = os.getenv('POSTGRES_USER')
# password = os.getenv('POSTGRES_PASSWORD') # set in .env file
# password = 'changeme'
uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'
uri

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(uri, echo=False)

In [ ]:
# df.to_sql('ads', engine)

In [ ]:
query_tmpl = "INSERT INTO ads (%s) VALUES (%s)"

from sqlalchemy import insert
from sqlalchemy import Table
from sqlalchemy import MetaData


# colors = [cm(1) if c < 0 else cm(0)
#               for c in coef[interesting_coefficients]]

table = Table('ads', MetaData(engine), autoload_with=engine)

for ad in ads:
#     values = ','.join(['\'' + str(v) + '\'' for v in ad.values()])
#     keys = ','.join(['"' + str(v) + '"' for v in ad.keys()])
    with engine.connect() as conn:
#         query = query_tmpl % (keys, values)
#         print(query)
#         result = conn.execute(query)
#         print(result)

        stmt = insert(table).values(ad)
        result = conn.execute(stmt)
        print(result)
#         conn.commit()